In [1]:
import torch

In [2]:
import numpy as np
torch.initial_seed()
torch.backends.cudnn.deterministic = True
torch.manual_seed(999)

In [3]:
data=[('What is your name?','my name is ravin'),
      ('Where do you live?','i live in mumbai'),
      ('how are you?','i am fine'),
      ('What is your favourite color','pink'),
      ('how old are you?','i am 27 years old'),
      ('What is your fathers profession?','he is a teacher'),
      ('What is your job?','i am an engineer')
      
]

In [4]:
question=[]
answer=[]
for text in data:
  question.append(text[0])
  answer.append(text[1])

In [5]:
ques_enc={}
i=1
max_ques=0
for q in question:
   words=q.split(' ')
   if(len(words)>max_ques):
     max_ques=len(words)
   for w in words:
     if w not in ques_enc:
       ques_enc[w]=i
       i+=1
max_ques

5

In [6]:
ans_enc={}
i=1
max_ans=0
for a in answer:
   words=a.split(' ')
   if(len(words)>max_ans):
     max_ans=len(words)
   for w in words:
     if w not in ans_enc:
       ans_enc[w]=i
       i+=1

In [7]:
input_dim=len(ques_enc)
output_dim=len(ans_enc)

In [8]:
start='<sos>'
end='<eos>'
pad='<pad>'

In [9]:
enc_ques={}
enc_ques[start]=0
enc_ques[pad]=input_dim+1
enc_ques[end]=input_dim+2

In [10]:
enc_ans={}
enc_ans[start]=0
enc_ans[pad]=output_dim+1
enc_ans[end]=output_dim+2

In [11]:
input_vect_dim=input_dim+3
output_vect_dim=output_dim+3
hidden_dim=input_dim

In [12]:
def generate_seq(text,dim,seq_len,word_code):
  input_seq=[]
  print(dim)
  for q in text:
    words=q.split(' ')
    enc=[]
    vect=np.zeros(dim)
    if dim==input_vect_dim:
      vect[enc_ques[start]]=1
      enc.append(vect)
    else:
      vect[enc_ans[start]]=1
      enc.append(vect)
    #print(len(words))
    for i,w in enumerate(words):
      vect=np.zeros(dim)
      vect[word_code[w]]=1
      enc.append(vect)
    #print('i=',i)
    #print(i+1,max_ques)
    for k in range(i+1,seq_len):
      v=np.zeros(dim)
      if dim==input_vect_dim:
        v[enc_ques[pad]]=1
        enc.append(v)
      else:
        v[enc_ans[pad]]=1
        enc.append(v)
    vect=np.zeros(dim)
    if dim==input_vect_dim:
      vect[enc_ques[end]]=1
    else:
      vect[enc_ans[end]]=1
    enc.append(vect)
    print(q,enc)
    input_seq.append(enc)
  Input_seq=np.array(input_seq)
  print(Input_seq.shape)
  return Input_seq

In [13]:
Input_seq=generate_seq(question,input_vect_dim,max_ques,ques_enc)
Input=torch.FloatTensor(Input_seq)
Input.shape

20
What is your name? [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1.])]
Where do you live? [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.]

torch.Size([7, 7, 20])

In [14]:
Output_seq=generate_seq(answer,output_vect_dim,max_ans,ans_enc)
Output=torch.FloatTensor(Output_seq)
Output.shape

22
my name is ravin [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1.])]
i live in mumbai [array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.]), array([0., 0., 0., 0., 0.

torch.Size([7, 7, 22])

In [15]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F

In [16]:
class Transliteration_EncoderDecoder(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size, verbose=False):
        super(Transliteration_EncoderDecoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        print(output_size)
        self.encoder_rnn_cell = nn.GRU(input_size, hidden_size)
        self.decoder_rnn_cell = nn.GRU(output_size,hidden_size)
        
        self.h2o = nn.Linear(hidden_size, output_size)
        self.init_output=torch.zeros(7,1,self.output_size)
        self.init_output[0,:,0]=1
        
        self.verbose = verbose
        
    def forward(self, input):
        
        # encoder
        #print(output.shape)
        input=input.view((input.shape[0],1,input.shape[1]))
        
        out, hidden = self.encoder_rnn_cell(input)
        hidden_dec=hidden
        #print(out.shape,hidden.shape)
        #print(self.init_output.shape)
        
        self.init_output,_=self.decoder_rnn_cell(self.init_output,hidden_dec)
        #print(self.init_output.shape)
        target=self.h2o(self.init_output)
        #print(target)
        target1=torch.zeros(7,1,self.output_size)
        for i,t in enumerate(target):
          if i==0:
            target1[:,:,i]=1
          else:
            target1[i,:,:]=target[i-1,:,:]
        self.init_output=target1
        #print('target:',target)
        #print('target1:',target1)
        target=target.view((target.shape[0],target.shape[2]))  
        target=F.log_softmax(target,dim=1) 
        #print(torch.argmax(out_dec,dim=1).shape)
        #print(torch.argmax(out_dec,dim=1))
        return target

In [17]:
model= Transliteration_EncoderDecoder(input_vect_dim,hidden_dim,output_vect_dim)

22


In [18]:
for i in range(3):
  print(model(Input[i]))
  print('---------')

tensor([[-2.8678, -3.2539, -3.2127, -3.2476, -3.0657, -3.0034, -3.2067, -3.2117,
         -2.9446, -2.9457, -3.2194, -3.0515, -3.1038, -3.1218, -3.2247, -3.0005,
         -2.8723, -3.3176, -3.0391, -3.3255, -3.0425, -2.9351],
        [-2.9204, -3.2568, -3.2152, -3.2924, -3.0607, -3.0200, -3.2737, -3.2149,
         -2.9803, -2.9456, -3.2407, -2.9854, -3.1081, -3.1383, -3.2049, -2.9170,
         -2.8585, -3.2674, -3.0591, -3.3000, -3.0030, -2.9572],
        [-2.9476, -3.2550, -3.2159, -3.3175, -3.0739, -3.0295, -3.3148, -3.2178,
         -3.0053, -2.9469, -3.2619, -2.9443, -3.1106, -3.1366, -3.1940, -2.8755,
         -2.8385, -3.2532, -3.0663, -3.2923, -2.9768, -2.9688],
        [-2.9617, -3.2504, -3.2153, -3.3313, -3.0895, -3.0339, -3.3380, -3.2208,
         -3.0190, -2.9492, -3.2782, -2.9206, -3.1114, -3.1297, -3.1891, -2.8557,
         -2.8223, -3.2518, -3.0697, -3.2895, -2.9603, -2.9754],
        [-2.9689, -3.2453, -3.2145, -3.3390, -3.1024, -3.0357, -3.3504, -3.2236,
         -3.025

In [19]:
def code2seq(code,text_type):
  if text_type=='ans':
    Enc=ans_enc
  else:
    Enc=ques_enc
  text=''
  for c in code:
    for key in Enc.keys():
      if Enc[key]==c:
        text=text+' '+key
  return text



In [22]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
epochs =50
model.train()
for i in range(epochs):
    Loss=0
    correct=0
    model.zero_grad()
    for (input_text,output_text,q,a) in zip(Input,Output,question,answer):
        #print(seq.shape)
        
        optimizer.zero_grad()
        out=model(input_text)
        #print(out.shape)
        out_true=torch.argmax(output_text,dim=1)
        #print(out_true)
        y_pred=torch.argmax(out,dim=1) 
        ques=torch.argmax(input_text,dim=1)
        #print('ques',code2seq(ques,'ques'))
        #print('ans',code2seq(y_pred,'ans'))
        #print(y_pred)
        #print('actual ans',code2seq(out_true,'ans'))
        if torch.equal(y_pred,out_true):
          correct+=1
        loss= loss_function(out,out_true)
        #loss=Variable(single_loss,requires_grad=True)
        loss.backward(retain_graph=True)
        Loss+=loss.item()
        #print(loss.item()/100)
        #print('**********************')
        optimizer.step()
    print('epochs: {}---loss: {}--acc:{}'.format(i,Loss,correct/10))
    

epochs: 0---loss: 5.534053146839142--acc:0.1
epochs: 1---loss: 4.364450842142105--acc:0.3
epochs: 2---loss: 4.413447767496109--acc:0.4
epochs: 3---loss: 3.734996497631073--acc:0.2
epochs: 4---loss: 3.26523394882679--acc:0.3
epochs: 5---loss: 3.0518191754817963--acc:0.2
epochs: 6---loss: 2.8304440826177597--acc:0.4
epochs: 7---loss: 2.6324123442173004--acc:0.6
epochs: 8---loss: 2.4166130274534225--acc:0.6
epochs: 9---loss: 2.233181908726692--acc:0.6
epochs: 10---loss: 2.081766188144684--acc:0.6
epochs: 11---loss: 1.949824720621109--acc:0.6
epochs: 12---loss: 1.8280886709690094--acc:0.6
epochs: 13---loss: 1.7152059897780418--acc:0.6
epochs: 14---loss: 1.6110580414533615--acc:0.6
epochs: 15---loss: 1.5586915388703346--acc:0.6
epochs: 16---loss: 1.533575415611267--acc:0.6
epochs: 17---loss: 1.4523241072893143--acc:0.6
epochs: 18---loss: 1.3791361898183823--acc:0.6
epochs: 19---loss: 1.2617467567324638--acc:0.6
epochs: 20---loss: 1.1831544488668442--acc:0.6
epochs: 21---loss: 1.127676602452

In [ ]:
for (input_text,output_text,q,a) in zip(Input,Output,question,answer):
        #print(seq.shape)
        out=model(input_text)
        #print(out.shape)
        out_true=torch.argmax(output_text,dim=1)
        #print(out_true)
        y_pred=torch.argmax(out,dim=1) 
        ques=torch.argmax(input_text,dim=1)
        print('ques',code2seq(ques,'ques'))
        print('ans',code2seq(y_pred,'ans'))